This notebook examines the two pickles provided by Keunwoo Choi (MSD_id_to_7D_id.pkl and 7D_id_to_path.pkl), as well as a few of the databases provided by the Million Song Dataset (lastfm_similars.db, track_metadata.db, lastfm_tags.db).

It then creates three important data structures for the machine learning task:

1) a set of valid track_ids, representing the only tracks for which we have song waveform data

2) a list of pairs of tracks, along with their similarities, where both tracks in the pair are in {valids}

3) a dictionary containing all the metadata for each song in {valids}

In [1]:
import os, sys
import sqlite3
import pickle
from collections import defaultdict

sys.path.insert(0, './deep-learning-models')
import audio_conv_utils

Using TensorFlow backend.


In [2]:
msd_to_7d = pickle.load(open('./code-shsd/MSD_id_to_7D_id.pkl', 'rb'))
_7d_to_path = pickle.load(open('./code-shsd/7D_id_to_path.pkl', 'rb'))

In [3]:
print(len(msd_to_7d))
print(len(_7d_to_path))

1000000
4598


In [4]:
# build dictionary of 7D to path here (for SHS songs, these are not included in the loaded dictioanry)

# create reverse msd_to_7d dictionary (to make sure all SHS songs are in the msd_to_7d dictionary)
_7d_to_msd = {}
for k,v in msd_to_7d.items():
    _7d_to_msd[v] = k

root_directory = "/mnt/kahuna/MSD_audio/SHSDataset/mp3"
SHS_7D_id_to_path = {}
for dir_name, subdir_list, file_list in os.walk(root_directory):
    for fname in file_list:
        path = os.path.abspath(os.path.join(dir_name, fname))
        relative_path = path[path.find('mp3/') + len('mp3/') : len(path)]
        if not str.isnumeric(relative_path[0]):
            continue
        _7_id = relative_path[relative_path.rfind('/') + 1 : relative_path.find('.clip.')]
        if _7_id not in _7d_to_msd:
            continue
        if _7_id not in SHS_7D_id_to_path:
            SHS_7D_id_to_path[_7_id] = 'SHSDataset/mp3/' + relative_path

In [5]:
# merge the two dictionaries
for k,v in _7d_to_path.items():
    if len(k) != 0 and k not in SHS_7D_id_to_path:
        SHS_7D_id_to_path[k] = 'songs/' + v

In [6]:
# rename
_7d_to_path = SHS_7D_id_to_path

In [7]:
# new lengths
print(len(msd_to_7d))
print(len(_7d_to_path))

1000000
22678


In [8]:
# make sure everything in _7d_to_path is in msd_to_7d
for k,v in _7d_to_path.items():
    if k not in _7d_to_msd:
        print("error:",len(k)," | ",v)

In [9]:
lastfm_similars_conn = sqlite3.connect('lastfm_similars.db')
lastfm_similars = lastfm_similars_conn.cursor()
track_metadata_conn = sqlite3.connect('track_metadata.db')
track_metadata = track_metadata_conn.cursor()
lastfm_tags_conn = sqlite3.connect('lastfm_tags.db')
lastfm_tags = lastfm_tags_conn.cursor()

In [10]:
# list all the tables
lastfm_similars.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
track_metadata.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
lastfm_tags.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
lastfm_tables = lastfm_similars.fetchall()
track_tables = track_metadata.fetchall()
lastfm_tags_tables = lastfm_tags.fetchall()
print(lastfm_tables)
print(track_tables)

[('similars_dest',), ('similars_dest_tmp',), ('similars_src',)]
[('songs',)]


In [11]:
# print the columns of these two tables
lastfm_columns = lastfm_similars.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'similars_src' AND type = 'table'")
track_columns = track_metadata.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'songs' AND type = 'table'")
lastfm_tags_columns = lastfm_tags.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'tags' AND type = 'table'")
print(lastfm_columns.fetchall())
print(track_columns.fetchall())

[('CREATE TABLE similars_src (tid TEXT, target TEXT)',)]
[('CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int, track_7digitalid int, shs_perf int, shs_work int)',)]


In [12]:
# grab all data from one particular table
lastfm_data = lastfm_similars.execute("SELECT * FROM similars_src")
track_data = track_metadata.execute("SELECT * FROM songs")
lastfm_all_tags = lastfm_tags.execute("SELECT tag FROM tags")

In [13]:
count = 0
for row in track_data:
    print(row)
    count +=1
    if count > 0:
        break

('TRMMMYQ128F932D901', 'Silent Night', 'SOQMMHC12AB0180CB8', 'Monster Ballads X-Mas', 'ARYZTJS1187B98C555', '357ff05d-848a-44cf-b608-cb34b5701ae5', 'Faster Pussy cat', 252.05506, 0.649822100201, 0.394031892714, 2003, 7032331, -1, 0)


In [14]:
count = 0
found = 0
for row in lastfm_data:
    if row[0] in msd_to_7d:
        if msd_to_7d[row[0]] in _7d_to_path:
            print(_7d_to_path[msd_to_7d[row[0]]])
            found += 1
    count +=1
    if count > 5:
        break
print(found)

0


In [15]:
count = 0
for row in lastfm_all_tags:
    print(row)
    count +=1
    if count > 9:
        break

('classic rock',)
('Progressive rock',)
('blues',)
('memphis slim',)
('pop',)
('70s',)
('Middle of the road',)
('Bonjour ca va',)
('Tony Levin',)
('instrumental',)


In [16]:
# generate 3 data structures:
# a list of all MSD_ids in the mp3 dataset
# a list of track pairs along with their distances, where both songs in the pair are present in the mp3 dataset
# a dictionary that maps the valid MSD_ids to (path_name, track_metadata)
valids = set()
pairs = []
msd_to_info = {}

# to build valids, get intersection of _7d_to_path and msd_to_7d: these are the songs we actually have data for
tmp = {}
count_tmp = defaultdict(int)
for k,v in _7d_to_path.items():
    count_tmp[k] += 1
    tmp[k] = []

#sanity check: no duplicates here
for k,v in count_tmp.items():
    if v > 1:
        print(k, v)
        break
else:
    print("no duplicates in _7d_to_path")

for k,v in msd_to_7d.items():
    if v in tmp:
        tmp[v].append(k)
        valids.add(k)

# there are duplicates in msd_to_7d, however
duplicates = []
for k,v in tmp.items():
    if len(v) > 1:
        duplicates.append((k,v))
    
print("number of valid entries in the msd_to_7d dict: ", len(valids))

# now assemble the track pairs
print("assembling track pairs...")
lastfm_data = lastfm_similars.execute("SELECT * FROM similars_src")
for row in lastfm_data:
    if row[0] in valids:
        song1_id = row[0]
        words = row[1].split(',')
        n = len(words)
        assert n % 2 == 0
        for i in range(int(n/2)):
            song2_id = words[i*2]
            if song2_id in valids:
                similarity = words[i*2+1]
                pairs.append((song1_id, song2_id, 1-float(similarity)))
                
# now get the dictionary map
print("creating dictionary map...")
track_data = track_metadata.execute("SELECT * FROM songs")
for row in track_data:
    if row[0] in valids:
        song_id = row[0]
        data = [_7d_to_path[msd_to_7d[row[0]]]]
        for elem in row[1:]:
            data.append(elem)
        msd_to_info[song_id] = data
        
# get all tids
print("getting all tids...")
lastfm_all_tracks = lastfm_tags.execute("SELECT tid FROM tids")
all_tids = []
for row in lastfm_all_tracks:
    all_tids.append(row[0])

# insert tags into msd_to_info
print("adding song tags to dictionary map...")
for tid in all_tids:
    if tid in valids:
        query = "SELECT tags.tag, tid_tag.val FROM tid_tag, tids, tags WHERE tags.ROWID=tid_tag.tag AND tid_tag.tid=tids.ROWID and tids.tid='%s'" % tid
        res = lastfm_tags.execute(query)
        valid_tags = []
        for row in res:
            if row[0] in audio_conv_utils.TAGS:
                valid_tags.append(row)
        assert tid in msd_to_info
        msd_to_info[tid].append(tuple(valid_tags))
    
# turn all dictionary elements into immutable tuples
print("converting all dictionary elements into tuples...")
for k,v in msd_to_info.items():
    msd_to_info[k] = tuple(v)

no duplicates in _7d_to_path
number of valid entries in the msd_to_7d dict:  22680
assembling track pairs...
creating dictionary map...
getting all tids...
adding song tags to dictionary map...
converting all dictionary elements into tuples...


In [17]:
print("valids size:", sys.getsizeof(valids)/1000000, "mB")
print("pairs size:", sys.getsizeof(pairs)/1000000, "mB")
print("msd_to_info size:", sys.getsizeof(msd_to_info)/1000000, "mB")

valids size: 2.097376 mB
pairs size: 3.012904 mB
msd_to_info size: 1.57296 mB


In [18]:
pickle.dump(valids, open("valids.pkl", "wb"))
pickle.dump(pairs, open("pairs.pkl", "wb"))
pickle.dump(msd_to_info, open("msd_to_info.pkl", "wb"))
print("saved to pickles")

saved to pickles
